In [1]:
import pandas as pd


In [2]:

# Load the data into a Pandas DataFrame
df = pd.read_excel('/Users/darpanchoudhary128/Desktop/ElectricVehicle_Population_Analysis/Electric_Vehicle_Population_Data.xlsx')


In [3]:
# Display the first few rows of the DataFrame
print(df.head())

   VIN (1-10)    County     City State  Postal Code  Model Year    Make  \
0  5YJXCAE26J    Yakima   Yakima    WA      98908.0        2018   TESLA   
1  JHMZC5F37M    Kitsap  Poulsbo    WA      98370.0        2021   HONDA   
2  5YJ3E1EB0K      King  Seattle    WA      98199.0        2019   TESLA   
3  1N4AZ0CP5D      King  Seattle    WA      98119.0        2013  NISSAN   
4  5YJSA1E21H  Thurston    Lacey    WA      98516.0        2017   TESLA   

     Model                   Electric Vehicle Type  \
0  MODEL X          Battery Electric Vehicle (BEV)   
1  CLARITY  Plug-in Hybrid Electric Vehicle (PHEV)   
2  MODEL 3          Battery Electric Vehicle (BEV)   
3     LEAF          Battery Electric Vehicle (BEV)   
4  MODEL S          Battery Electric Vehicle (BEV)   

  Clean Alternative Fuel Vehicle (CAFV) Eligibility  Electric Range  \
0           Clean Alternative Fuel Vehicle Eligible             238   
1           Clean Alternative Fuel Vehicle Eligible              47   
2          

In [4]:
# Check for missing values
print(df.isnull().sum())


VIN (1-10)                                             0
County                                                 3
City                                                   3
State                                                  0
Postal Code                                            3
Model Year                                             0
Make                                                   0
Model                                                222
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                 305
DOL Vehicle ID                                         0
Vehicle Location                                      33
Electric Utility                                       3
2020 Census Tract                                      3
dtype: int64


In [7]:
# Example: Remove duplicate rows
df = df.drop_duplicates()

In [9]:
# Example: Convert 'Model_Year' to datetime
df['Model Year'] = pd.to_datetime(df['Model Year'], format='%Y')


In [11]:
# Example: Convert 'Postal_Code' to string
df['Postal Code'] = df['Postal Code'].astype(str)

In [12]:
# Example: Remove duplicate rows
df = df.drop_duplicates()

In [14]:

# Calculate the Total Base MSRP and Average Electric Range for Each Make and Model Combination
model_stats = df.groupby(['Make', 'Model']).agg(
    ModelCount=('VIN (1-10)', 'count'),
    TotalBaseMSRP=('Base MSRP', 'sum'),
    AvgElectricRange=('Electric Range', 'mean')
).sort_values(by=['ModelCount', 'TotalBaseMSRP'], ascending=[False, False])


In [15]:
print(model_stats.head())


                   ModelCount  TotalBaseMSRP  AvgElectricRange
Make      Model                                               
TESLA     MODEL 3       25311              0        131.567303
          MODEL Y       22078              0         31.356509
NISSAN    LEAF          12961              0         88.033562
TESLA     MODEL S        7399      106895300        186.946614
CHEVROLET BOLT EV        5335              0        161.173196


In [16]:
# Identify Models with the Highest Total Base MSRP and Average Electric Range
top_models = model_stats.nlargest(5, columns=['TotalBaseMSRP', 'AvgElectricRange'])
print(top_models)


               ModelCount  TotalBaseMSRP  AvgElectricRange
Make  Model                                               
TESLA MODEL S        7399      106895300        186.946614
BMW   530E            376       15498300         15.678191
KIA   SOUL            423       12843900         93.000000
VOLVO XC60            858       11267700         23.095571
BMW   330E            397        8268100         17.607053


In [18]:
# Calculate the Percentage of Electric Vehicles Eligible for Clean Alternative Fuel by County
county_eligibility_percentage = df.groupby('County').agg(
    TotalVehicles=('VIN (1-10)', 'count'),
    EligibleVehicles=('Clean Alternative Fuel Vehicle (CAFV) Eligibility', lambda x: (x == 'Clean Alternative Fuel Vehicle Eligible').sum()),
    PercentageEligible=('Clean Alternative Fuel Vehicle (CAFV) Eligibility', lambda x: (x == 'Clean Alternative Fuel Vehicle Eligible').mean() * 100)
).sort_values(by='PercentageEligible', ascending=False)


In [19]:
print(county_eligibility_percentage.head())


            TotalVehicles  EligibleVehicles  PercentageEligible
County                                                         
Yavapai                 1                 1               100.0
Oldham                  1                 1               100.0
Sarasota                1                 1               100.0
DeKalb                  1                 1               100.0
Santa Cruz              1                 1               100.0


In [21]:
# Find the Top 5 Legislative Districts with the Highest Average Electric Range
top_legislative_districts = df.groupby('Legislative District')['Electric Range'].mean().nlargest(5)
print(top_legislative_districts)


Legislative District
45.0    86.748954
41.0    86.656396
12.0    84.751278
48.0    84.543599
40.0    82.369159
Name: Electric Range, dtype: float64


In [22]:
# Identify Models and Makes with Low Battery Range but High Base MSRP
low_battery_high_msrp = df[(df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Not eligible due to low battery range') & (df['Base MSRP'] > 50000)]
print(low_battery_high_msrp[['Make', 'Model', 'Base MSRP', 'Electric Range']])


           Make    Model  Base MSRP  Electric Range
277         BMW     530E      54950              14
379       VOLVO     XC60      52900              17
576       VOLVO     XC60      52900              17
664     PORSCHE  CAYENNE      81100              14
1000      VOLVO     XC60      52900              17
...         ...      ...        ...             ...
129694      BMW     530E      52650              16
129775    VOLVO     XC60      52900              17
130011    VOLVO     XC60      52900              17
130198      BMW     740E      89100              14
130377      BMW     530E      53400              16

[640 rows x 4 columns]
